In [1]:
import json
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [2]:
f = open('testnet-results-new.json')
data = json.load(f)
df_test = pd.json_normalize(data)
f.close()

f = open('sim-results.json')
data = json.load(f)
df_sim = pd.json_normalize(data)
f.close()

In [3]:
df_test['marketCap'] = df_test['marketCap'].map(lambda x: x/1e18)
df_test['price'] = df_test['price'].map(lambda x: round(x/1e18,2))
df_test['lowWall'] = df_test['lowWall'].map(lambda x: round(x/1e18,2))
df_test['highWall'] = df_test['highWall'].map(lambda x: round(x/1e18,2))
df_test['lowCushion'] = df_test['lowCushion'].map(lambda x: round(x/1e18,2))
df_test['highCushion'] = df_test['highCushion'].map(lambda x: round(x/1e18,2))
df_test['lowCapacity'] = df_test['lowCapacity'].map(lambda x: x/1e18)
df_test['highCapacity'] = df_test['highCapacity'].map(lambda x: x/1e9) * df_test['highWall']
df_test['reserves'] = df_test['reserves'].map(lambda x: x/1e18)
df_test['liqRatio'] = df_test['liqRatio'].map(lambda x: x/100)
df_test['supply'] = df_test['supply'].map(lambda x: x/1e9)
df_test['day'] = df_test['epoch'].map(lambda x: x//3)

rows = [i for i in range (1, 1095)]
keep = [i for i in range (0, 1095, 3)]
drop = list(set(rows) - set(keep))

df_testnet = df_test.drop(df_test.index[drop])
df_testnet = df_testnet.set_index(['day'])
df_testnet = df_testnet.drop(columns=['key','epoch','seed'])

df_testnet2 = df_testnet[1:]
df_testnet2 = df_testnet2.reset_index()
df_testnet2['day'] = df_testnet2['day'].map(lambda x: x-2)

df_testnet2.head()

,day,marketCap,price,reserves,liqRatio,supply,lowCapacity,highCapacity,lowWall,highWall,lowCushion,highCushion
0,-1,3.474903e+08,13.83,2.751232e+08,9.13,2.511881e+07,1.875000e+07,2.923774e+07,10.20,18.13,12.04,16.29
1,0,3.747596e+08,14.87,2.776565e+08,9.96,2.520829e+07,1.875000e+07,2.925387e+07,10.20,18.14,12.05,16.30
2,1,4.430318e+08,17.36,2.757731e+08,9.34,2.551440e+07,1.875000e+07,2.551829e+07,10.28,18.27,12.14,16.42
3,2,2.637977e+08,10.30,2.716855e+08,7.98,2.560529e+07,1.875000e+07,2.542052e+07,10.24,18.20,12.09,16.35
4,3,2.734117e+08,10.71,2.696825e+08,7.94,2.553278e+07,1.701477e+07,2.521101e+07,10.15,18.05,11.99,16.22


In [4]:
df_sim = df_sim.drop(columns=['seed','day'])
df_sim['price'] = df_sim['price'].apply(lambda x: round(float(x),2))
df_sim['netFlow'] = df_sim['netFlow'].apply(lambda x: round(float(x),2))
df_sim['lowCushion'] = df_sim['lowCushion'].apply(lambda x: round(float(x),2))
df_sim['lowWall'] = df_sim['lowWall'].apply(lambda x: round(float(x),2))
df_sim['highCushion'] = df_sim['highCushion'].apply(lambda x: round(float(x),2))
df_sim['highWall'] = df_sim['highWall'].apply(lambda x: round(float(x),2))
df_sim['reservesUSD'] = df_sim['reservesUSD'].apply(lambda x: round(float(x),2))
df_sim['liqUSD'] = df_sim['liqUSD'].apply(lambda x: round(float(x),2))
df_sim['treasury'] = df_sim['treasury'].apply(lambda x: round(float(x),2))
df_sim['marketcap'] = df_sim['marketcap'].apply(lambda x: round(float(x),2))
df_sim['supply'] = df_sim['supply'].apply(lambda x: round(float(x),2))
df_sim['askCapacity'] = df_sim['askCapacity'].apply(lambda x: round(float(x),2)) * df_sim['highWall']
df_sim['askCapacityTarget'] = df_sim['askCapacityTarget'].apply(lambda x: round(float(x),2)) * df_sim['highWall']
df_sim['bidCapacity'] = df_sim['bidCapacity'].apply(lambda x: round(float(x),2))
df_sim['bidCapacityTarget'] = df_sim['bidCapacityTarget'].apply(lambda x: round(float(x),2))
df_sims = df_sim[['marketcap', 'price', 'reservesUSD', 'netFlow', 'lowWall', 'lowCushion', 'highCushion', 'highWall', 'askCapacity', 'askCapacityTarget', 'bidCapacity', 'bidCapacityTarget']]
df_sims.head()

,marketcap,price,reservesUSD,netFlow,lowWall,lowCushion,highCushion,highWall,askCapacity,askCapacityTarget,bidCapacity,bidCapacityTarget
0,3.530000e+08,14.12,2.500000e+08,-195578.00,10.20,12.04,16.30,18.14,2.925387e+07,2.925387e+07,18750000.00,18750000.00
1,3.787287e+08,15.13,2.500000e+08,972267.06,9.83,11.60,15.70,17.47,2.817338e+07,2.817338e+07,18750000.00,18750000.00
2,4.367377e+08,17.47,2.536704e+08,5704737.98,9.83,11.60,15.70,17.47,2.450323e+07,2.924808e+07,18750000.00,18750000.00
3,2.984348e+08,11.63,2.524110e+08,-6334152.66,9.85,11.63,15.74,17.51,2.455933e+07,2.974544e+07,17490579.19,19025279.69
4,3.081651e+08,12.03,2.524110e+08,476650.43,9.93,11.73,15.86,17.66,2.476972e+07,2.977771e+07,18930823.13,18930823.13


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['price'], name='price (sims)'))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['lowWall'], name='lowWall (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['highWall'], name='highWall (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['lowCushion'], name='lowCushion (sims)', line=dict(color='grey', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['highCushion'], name='highCushion (sims)', line=dict(color='grey', dash='dot')))

fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['price'], name='price (testnet)'))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowWall'], name='lowWall (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highWall'], name='highWall (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowCushion'], name='lowCushion (testnet)', line=dict(color='silver', dash='dot')))
fig.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highCushion'], name='highCushion (testnet)', line=dict(color='silver', dash='dot')))

fig.show()

In [6]:
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['askCapacity'], name='askCapacity (sims)', line=dict(color='#009aff')))
fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['askCapacityTarget'], name='askCapacityTarget (sims)', line=dict(color='grey', dash='dot')))

fig2.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['highCapacity'], name='askCapacity (testnet)', line=dict(color='#ff334e')))

fig2.show()

In [7]:
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['bidCapacity'], name='bidCapacity (sims)', line=dict(color='#565bff')))
fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_sims['bidCapacityTarget'], name='bidCapacityTarget (sims)', line=dict(color='grey', dash='dot')))

fig3.add_trace(go.Scatter(x=df_testnet2['day'], y=df_testnet2['lowCapacity'], name='bidCapacity (testnet)', line=dict(color='#ff59a2')))

fig3.show()

In [8]:
fig4 = pd.DataFrame()
fig4['supply (sims)'] = df_sim['supply']
fig4['supply (testnet)'] = df_testnet2['supply'][:-1]
fig4.plot()